In [7]:
import pandas as pd
import re
import nltk
nltk.download('all')
from nltk.corpus import stopwords

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_r

In [8]:
url = "https://raw.githubusercontent.com/romauligraciella/sirekap_sentiment_analysis/main/scraping/Review_SIREKAP.csv"
dataset = pd.read_csv(url, usecols=["userName", "score",	"content"])

In [9]:
## Import kamus untuk mengubah singkatan
kamus_alay = pd.read_csv('https://raw.githubusercontent.com/nasalsabila/kamus-alay/master/colloquial-indonesian-lexicon.csv')

## Membuat dictionary untuk memetakan singkatan dari kata
nor_dict = {}
for index, row in kamus_alay.iterrows():
  if row[0] not in nor_dict:
    nor_dict[row[0]] = row[1]

def clean_text_1(text):
    # 01 Konversi ke huruf kecil semua
    text = text.strip().lower()
    # 02 Menghilangkan tautan www.* atau https?://*
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',text)
    # 03 Menghilangkan @username
    text = re.sub('@[^\s]+','',text)
    # 04 Menghilangkan # dari #kata
    text = re.sub(r'#([^\s]+)', r'\1', text)
    # 05 Menghilangkan spasi berlebihan
    text = re.sub('[\s]+', ' ', text)
    # 06 Menghilangkan emoticon dan semacamnya
    emoji_pattern = re.compile("["
                                u"\U0001F600-\U0001F64F"  # emoticons
                                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)

    # Pembersihan kata
    words = text.split()
    tokens=[]
    for ww in words:
        # Memisahkan kata berulang
        for w in re.split(r'[-/\s]\s*', ww):
            # Menghapus huruf berulang yang lebih dari dua kali
            pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
            w = pattern.sub(r"\1\1", w)
            w = w.strip('\'"?,.')
            # Memeriksa apakah suatu kata terbentuk dari minimal dua huruf
            val = re.search(r"^[a-zA-Z][a-zA-Z][a-zA-Z]*$", w)
            if w in nor_dict:
                w = nor_dict[w]

            if w == "rt" or val is None:
                continue
            else:
                tokens.append(w.lower())

    text = " ".join(tokens)

    return text.strip()

<ipython-input-9-2acac2d563ba>:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[0] not in nor_dict:
<ipython-input-9-2acac2d563ba>:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  nor_dict[row[0]] = row[1]


In [10]:
## Untuk cTFIDF
def clean_text_2(text):
    # 01 Menghilangkan tanda baca
    text = re.sub(r'[^\w\s]',' ', text)
    # 02 Menghilangkan angka
    text = re.sub("[0-9]","", text)

    return text.strip()


additional_stopwords = ['nya', 'ya', 'kalo', 'tolong', 'kali', 'sih', 'gue',
                          'gua', 'kasih', 'terima', 'terimakasih', 'min', 'pas',
                          'banget', 'saya', 'saat', 'deh', 'dll', 'nih', 'donk',
                          'ny', 'si', 'eh', 'tu', 'tuh', 'mah', 'loh', 'aja']
stop_words = stopwords.words('indonesian') + additional_stopwords + stopwords.words('english')
stop_words = set(stop_words)

def remove_stop_words(s):
    return ' '.join(word for word in s.split() if word not in stop_words)

In [11]:
dataset["text_1"] = dataset["content"].astype(str)
dataset["text_1"] = dataset["text_1"].map(lambda x: clean_text_1(x))
dataset = dataset[~dataset['text_1'].str.contains('game|hero|match| epep | player | ping | resolusi |patch|pay to win|mobile legend|bokep|analog| gem |sakitu| ora |konoha|keun|asayap|kaizo|sange|ngent|ike nurjanah|mabar|multiplayer| skin |lvl | legend |scater|coklat|ladusing|this|not good|no good|pintu| rank |jomblo| alok |cheat|repam|revam|gatcha|quest|leon|kipas| war |ranked|jerawat|rrq|karina|youtube|nerf|item senjata|e-sport|genshin|sensouman|tong fang|mangkono|balmond|gacha|battle|virtex|sotot|shotgun|taruno|edora|skin gratis|streaming|tiktok|edit wajah| bunga |watashi|genre|saikyo|free fire|freefire|tempur|skin gratis|gofood|pospay|porno|jessica|ranked|edora|karina| ff |npc|roket|fastron|kresek',
                                                           case=False)]

dataset["text_2"] = dataset["text_1"].map(lambda x: clean_text_2(x))
dataset["text_2"] = dataset["text_2"].apply(lambda x: remove_stop_words(x))
dataset = dataset[dataset["text_2"].apply(lambda x: len(x.split()) >=3)]

dataset = dataset.drop('content', axis=1)
dataset.head()

<ipython-input-11-f05efb418ec4>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["text_2"] = dataset["text_1"].map(lambda x: clean_text_2(x))
<ipython-input-11-f05efb418ec4>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["text_2"] = dataset["text_2"].apply(lambda x: remove_stop_words(x))


,userName,score,text_1,text_2
0,Pengguna Google,1,aplikasinya kurang stabil scanner camera juga ...,aplikasinya stabil scanner camera bagus suka n...
1,Pengguna Google,3,saya ikut ujibeban nasional terjadi masalah ke...,ujibeban nasional upload berkas persen salinan...
2,Pengguna Google,5,tolong untuk tambahkan fitur unduh data agar k...,tambahkan fitur unduh data sengaja kehapus dat...
3,Pengguna Google,1,apk yang versi ini kenapa malah susah sering m...,apk versi susah macet masuk profil kesalahan i...
4,Pengguna Google,2,di versi lancar dapat wa dari pps disuruh unin...,versi lancar wa pps disuruh uninstall install ...


In [12]:
dataset.to_csv("data_cleaned", index=False)